# Data preparation for the SDG Indicators

We use World's Bank data set: https://databank.worldbank.org/source/sustainable-development-goals-(sdgs)#

We aim to have one pandas data frame per country, with all indicators. We save them as separate csv files.

First load data and look at its structure.

In [1]:
import numpy as np
import pandas as pd
import math
import os
import pickle
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
import warnings
warnings.filterwarnings('ignore')

In [2]:
wb_data = pd.read_csv('/Users/saravallejomengod/MathsYear4/M4R/utils/Data_Extract_From_Sustainable_Development_Goals/SDG_data.csv', dtype=object)

In [3]:
wb_data

,Country Name,Country Code,Series Name,Series Code,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,Afghanistan,AFG,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,..,..,..,..,..,..,...,20.68,22.33,24.08,26.17,27.99,30.1,32.44,..,..,..
1,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,..,..,..,..,..,..,...,42.7,43.2220189082037,69.1,68.9332656860352,89.5,71.5,97.7,97.7,98.7132034301758,..
2,Afghanistan,AFG,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,..,..,..,..,..,..,...,30.2188002485248,29.5728807664918,60.8491567562828,61.2821986356521,86.5005119066394,64.573353938483,97.0993597954095,97.0919732441472,98.2728721849277,..
3,Afghanistan,AFG,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,..,..,..,..,..,..,...,82.8,86.5677791512155,95,92.6737670898438,98.7,92.5,99.5,99.5,100,..
4,Afghanistan,AFG,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,..,..,..,..,..,..,...,..,9.00501251220703,..,..,9.96100044250488,..,..,14.8933124542236,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101784,Data from database: Sustainable Development Go...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#delete 5 last rows that are blank or have other info
wb_data = wb_data[:-5] 
wb_data.tail()

,Country Name,Country Code,Series Name,Series Code,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
101776,Zimbabwe,ZWE,"Water productivity, total (constant 2010 US$ G...",ER.GDP.FWTL.M3.KD,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
101777,Zimbabwe,ZWE,Women Business and the Law Index Score (scale ...,SG.LAW.INDX,55,55,57.5,57.5,66.9,66.9,...,86.9,86.9,86.9,86.9,86.9,86.9,86.9,86.9,86.9,86.9
101778,Zimbabwe,ZWE,Women making their own informed decisions rega...,SG.DMK.SRCR.FN.ZS,..,..,..,..,..,..,...,..,58.8,..,..,..,59.9,..,..,..,..
101779,Zimbabwe,ZWE,Women who were first married by age 15 (% of w...,SP.M15.2024.FE.ZS,..,..,..,..,6.4,..,...,..,3.9,..,..,..,3.7,..,..,..,..
101780,Zimbabwe,ZWE,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS,..,..,..,..,31,..,...,..,30.5,..,..,33.5,32.4,..,..,..,..


In [5]:
#transform years in column index by year
columns = list(wb_data.columns)
for column in columns[4:]:
    columns.append(column[:4])
    columns.remove(column)

wb_data.columns = columns

years = columns[4:]

Have a look at different country names in our data set.

In [6]:
c = pd.read_csv('/Users/saravallejomengod/MathsYear4/M4R/utils/countries_wb.csv', dtype=str, delimiter=';', header=None)
countries = list(c[0])

len(countries)

181

In [7]:
countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt, Arab Rep.',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia, The',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hungary',
 'Ic

## SDG 13

For SDG13 we take data from UN SDG's data set. Available from: https://unstats.un.org/sdgs/indicators/database/

In [8]:
# UN data for SDG 13
SDG13_data = pd.read_csv('/Users/saravallejomengod/MathsYear4/M4R/utils/SDG13_data.csv', dtype=object)

In [9]:
SDG13_data.head()

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,13,13.1,13.1.1,VC_DSR_AFFCT,Number of people affected by disaster (number),4,Afghanistan,2017,29933.2,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,13.1,13.1.1,VC_DSR_AFFCT,Number of people affected by disaster (number),4,Afghanistan,2018,283200.12,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,13.1,13.1.1,VC_DSR_AFFCT,Number of people affected by disaster (number),4,Afghanistan,2019,332536.23324,2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,13.1,13.1.1,VC_DSR_AFFCT,Number of people affected by disaster (number),8,Albania,2005,29259.74973,2005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13,13.1,13.1.1,VC_DSR_AFFCT,Number of people affected by disaster (number),8,Albania,2006,1724.73852,2006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# check
SDG13_data['SeriesCode'].unique()

array(['VC_DSR_AFFCT', 'VC_DSR_DAFF', 'VC_DSR_IJILN', 'VC_DSR_MISS',
       'VC_DSR_MORT', 'VC_DSR_MTMN', 'VC_DSR_MTMP', 'VC_DSR_PDAN',
       'VC_DSR_PDLN', 'VC_DSR_PDYN', 'SG_DSR_LGRGSR', 'SG_DSR_SFDRR',
       'SG_DSR_SILN', 'SG_DSR_SILS', 'SG_GOV_LOGV'], dtype=object)

In [11]:
#Country Names in UN data
groupings_UN = list(SDG13_data['GeoAreaName'].unique())
groupings_UN

['Afghanistan',
 'Albania',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Australia',
 'Austria',
 'Bangladesh',
 'Armenia',
 'Barbados',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Botswana',
 'Brazil',
 'Belize',
 'Solomon Islands',
 'Bulgaria',
 'Myanmar',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Cabo Verde',
 'Sri Lanka',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czechia',
 'Dominica',
 'Ecuador',
 'El Salvador',
 'Equatorial Guinea',
 'Ethiopia',
 'Estonia',
 'Fiji',
 'Finland',
 'France',
 'Djibouti',
 'Georgia',
 'Gambia',
 'State of Palestine',
 'Ghana',
 'Grenada',
 'Guatemala',
 'Guyana',
 'Honduras',
 'Indonesia',
 'Iran (Islamic Republic of)',
 'Iraq',
 'Ireland',
 "Côte d'Ivoire",
 'Jamaica',
 'Japan',
 'Kazakhstan',
 'Jordan',
 'Kenya',
 'Republic of Korea',
 'Kosovo',
 'Kuwait',
 'Kyrgyzstan',
 "Lao People's Democratic Republic",
 'Lebanon',
 'Madagascar',
 'Malawi',
 'Malaysia',
 'Maldives',
 'Mali',


In [12]:
#Replace names by names in WB data set
SDG13_data.replace({"Republic of Korea": "Korea, Rep.", "Democratic People's Republic of Korea": 
                    "Korea, Dem. People's Rep.", 'Gambia': 'Gambia, The', 
                    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom', 'Congo': 'Congo, Rep.',
                    'Democratic Republic of the Congo': 'Congo, Dem. Rep.', 'Czechia': 'Czech Republic', 
                    'Iran (Islamic Republic of)': 'Iran, Islamic Rep.', "Côte d'Ivoire": "Cote d'Ivoire", 
                    'Kyrgyzstan': 'Kyrgyz Republic', "Lao People's Democratic Republic": 'Lao PDR', 
                    'Republic of Moldova': 'Moldova', 'Micronesia (Federated States of)': 'Micronesia, Fed. Sts.',
                    'Slovakia': 'Slovak Republic', 'Viet Nam': 'Vietnam', 'Egypt': 'Egypt, Arab Rep.', 
                    'United Republic of Tanzania': 'Tanzania','United States of America': 'United States', 
                    'Venezuela (Bolivarian Republic of)': 'Venezuela, RB', 'Yemen': 'Yemen, Rep.', 
                    'Bahamas': 'Bahamas, The', 'Bolivia (Plurinational State of)': 'Bolivia'}, inplace=True)
groupings = list(SDG13_data['GeoAreaName'].unique())

In [13]:
# list of keys to delete
delete_groups = []

for g in list(groupings):
    if g not in countries:
        delete_groups.append(g)
        
# delete
for dg in delete_groups:
    groupings.remove(dg)

delete_groups

['Cabo Verde',
 'Cook Islands',
 'State of Palestine',
 'Kosovo',
 'Marshall Islands',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Eswatini',
 'American Samoa',
 'French Polynesia',
 'New Caledonia',
 'Monaco',
 'World',
 'Australia and New Zealand',
 'Central and Southern Asia',
 'Sub-Saharan Africa',
 'Latin America and the Caribbean',
 'Europe and Northern America',
 'Oceania (exc. Australia and New Zealand)',
 'Northern Africa and Western Asia',
 'Eastern and South-Eastern Asia']

Let's now save a data frame with all of the meta-information. We delete the columns which are specific in area and time, and of course we do not want to have the values in this data frame. In the end, we delete all duplicate entries in the column SeriesCode. So, we are left with the information we wanted: mapping the series codes to the indicators, the Source for the data, the Units measured in, etc

In [14]:
#get info
info = SDG13_data.drop(columns=['GeoAreaCode', 'GeoAreaName', 'TimePeriod', 'Value', 'Time_Detail']).drop_duplicates(subset=['Indicator', 'SeriesCode'])

info.head()

,Goal,Target,Indicator,SeriesCode,SeriesDescription,TimeCoverage,UpperBound,LowerBound,BasePeriod,Source,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,13,13.1,13.1.1,VC_DSR_AFFCT,Number of people affected by disaster (number),NaN,NaN,NaN,NaN,United Nations Office for Disaster Risk Reduct...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1218,13,13.1,13.1.1,VC_DSR_DAFF,Number of directly affected persons attributed...,NaN,NaN,NaN,NaN,United Nations Office for Disaster Risk Reduct...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2436,13,13.1,13.1.1,VC_DSR_IJILN,Number of injured or ill people attributed to ...,NaN,NaN,NaN,NaN,United Nations Office for Disaster Risk Reduct...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3406,13,13.1,13.1.1,VC_DSR_MISS,Number of missing persons due to disaster (num...,NaN,NaN,NaN,NaN,United Nations Office for Disaster Risk Reduct...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3800,13,13.1,13.1.1,VC_DSR_MORT,Number of deaths due to disaster (number),NaN,NaN,NaN,NaN,United Nations Office for Disaster Risk Reduct...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# list of all series codes of SDG 13
seriescodes_13 = set(list(info['SeriesCode']))
seriescodes_13

{'SG_DSR_LGRGSR',
 'SG_DSR_SFDRR',
 'SG_DSR_SILN',
 'SG_DSR_SILS',
 'SG_GOV_LOGV',
 'VC_DSR_AFFCT',
 'VC_DSR_DAFF',
 'VC_DSR_IJILN',
 'VC_DSR_MISS',
 'VC_DSR_MORT',
 'VC_DSR_MTMN',
 'VC_DSR_MTMP',
 'VC_DSR_PDAN',
 'VC_DSR_PDLN',
 'VC_DSR_PDYN'}

In [16]:
# count how many we have
len(seriescodes_13)

15

In [18]:
SDG13_data.loc[SDG13_data['SeriesCode'] == 'SG_DSR_SFDRR']

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
10158,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,1,World,2019,85,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10159,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,53,Australia and New Zealand,2019,2,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10160,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,62,Central and Southern Asia,2019,8,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10161,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,202,Sub-Saharan Africa,2019,20,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10162,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,419,Latin America and the Caribbean,2019,9,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10163,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,513,Europe and Northern America,2019,25,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10164,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,543,Oceania (exc. Australia and New Zealand),2019,2,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10165,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,747,Northern Africa and Western Asia,2019,12,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10166,13,13.1,13.1.2,SG_DSR_SFDRR,Number of countries that reported having a Nat...,753,Eastern and South-Eastern Asia,2019,7,2015-2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We notice that 'SG_DSR_SFDRR' is binary, it indicates the "number of countries that reported having a National DRR Strategy which is aligned to the Sendai Framework" and we only have data for groupings of countries, ie. not for each country individually. Therefore, we delete this series code.

In [27]:
indexNames = SDG13_data[SDG13_data['SeriesCode'] == 'SG_DSR_SFDRR'].index
SDG13_data = SDG13_data.drop(indexNames)
indexNames2 = info[info['SeriesCode'] == 'SG_DSR_SFDRR'].index
info = info.drop(indexNames2)

seriescodes_13.remove('SG_DSR_SFDRR')

We convert the data set into multiple small data sets by creating a dictionary that contains the groupings' names as keys.

First, we create empty data frames for each key.

In [28]:
dict_13 = {c: pd.DataFrame() for c in countries}

In [29]:
# check, should be empty
#dict_all.get('Belize')
dict_13.get('Belize')

""


Second, we replace each of the empty data frames with the data we have available for them. Note, that our dictionary will be the ensamble of all groupings.

In [30]:
for c in countries:    # memory-intensive
    dict_13[c] = SDG13_data[SDG13_data['GeoAreaName'].isin(['{}'.format(c)])]

In [31]:
# check
dict_13['Azerbaijan']

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
9897,13,13.1,13.1.2,SG_DSR_LGRGSR,Score of adoption and implementation of nation...,31,Azerbaijan,2017,0,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we have one data frame per country. The next step is to have years as columns.

The next cell gives us the series codes in the rows and the years in the columns. These series codes are unique descriptions of the sub-indicators and we match these series codes to indicators and all other information in a different data frame.

In [32]:
for c in countries:
    if c not in groupings:
        print(c)
        dict_13[c] = pd.DataFrame(index=seriescodes_13, columns=years)
    else:
        dict_13[c] = dict_13.get(c).pivot_table(values='Value', index='SeriesCode', columns='TimePeriod', dropna=False, aggfunc='first')

Bahamas, The
Bahrain
Belgium
Benin
Brunei Darussalam
Canada
Chad
Congo, Dem. Rep.
Congo, Rep.
Cuba
Dominican Republic
Eritrea
Gabon
Greece
Haiti
Hungary
Iceland
India
Israel
Latvia
Lesotho
Libya
Lithuania
Luxembourg
Malta
Mauritania
Moldova
Nigeria
Oman
Puerto Rico
Sao Tome and Principe
Singapore
Suriname
Turkmenistan
Tuvalu
United Arab Emirates
Uzbekistan


In [33]:
dict_13.get('Albania')

TimePeriod,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
SeriesCode,,,,,,,,,,,,,,,
SG_DSR_LGRGSR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05,NaN
SG_DSR_SILN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
SG_DSR_SILS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.63934,NaN
SG_GOV_LOGV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,NaN
VC_DSR_AFFCT,29259.74973,1724.73852,2346.30497,1856.57818,10580.79152,90495.12157,5341.90169,6665.61371,16150.40793,3914.13062,1500.08626,146.19031,752.85467,4683,517.84045
VC_DSR_DAFF,947.89604,56.30841,77.3339,61.83064,355.89085,3069.6822,182.40456,228.73732,556.18413,135.14212,51.98172,5.08383,26.17177,163.15231,18.09096
VC_DSR_IJILN,11,3,2,3,3,1025,208,20,13,154,19,15,182,18,13
VC_DSR_MISS,NaN,NaN,NaN,NaN,NaN,NaN,8,1,NaN,NaN,2,NaN,NaN,NaN,NaN
VC_DSR_MORT,7,2,2,16,6,13,13,7,9,9,10,20,6,21,4


### Cleaning up and transforming all country data frames into the same dimensions
We have a couple of things to do to make our data frames workable:

1. We have some values in the data frames which we do not want, as e.g. ,,  = , N, etc. We replace them with appropriate values, i.e. 0, or simply a space.
2. Some data frames have data from 1990 to 2018, some others from 1992 to 2018. We want to have all data frames having data from 1990 to 2018, i.e. an equal amount of columns. The additional columns are filled with NaNs.
3. Some data frames have not all indicators and sub-indicators listed, but we would like to have all of them in all data frames. These additional rows are filled with NaNs.
Let's start with the first task, i.e. cleaning up the data frames.

We first need to define lists for all years, i.e. 1990 to 2018 and all indicators and sub-indicators, i.e. series codes.

Now, we insert the missing years for all groupings. We want to have NaNs in those columns.

In [34]:
# example
list(dict_13['Germany'])

['2015', '2016', '2017', '2018', '2019']

Firstly, we insert the missing years as columns for all groupings.

In [35]:
for c in countries:    # memory-intensive
    for year in years:
        if year not in list(dict_13[c]):
            dict_13[c]['{}'.format(year)] = np.nan
    # having the years in order
    dict_13[c] = dict_13[c][years]

In [36]:
# check
dict_13['Germany']

TimePeriod,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
SeriesCode,,,,,,,,,,,,,,,,,,,,,
SG_DSR_LGRGSR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.575,0.575,0.575,0.575,0.575
SG_DSR_SILN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11092,NaN,NaN,NaN,NaN
SG_DSR_SILS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN
SG_GOV_LOGV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11092,11059,11059,NaN,NaN


Secondly, we insert the missing series codes as rows. 
Let's have all $J$ sub-indicators we want for each country as rows. We fill these rows with NaNs.

In [37]:
len(list(dict_13['Nicaragua'].index))

10

In [38]:
for c in countries:
    for seriescode in seriescodes_13:
        if seriescode not in list(dict_13[c].index):
            dict_13[c].loc[seriescode] = np.nan    # fill these rows with NaNs

In [39]:
# check: do we have J many?
len(list(dict_13['Nicaragua'].index))

14

In [40]:
# convert all to floats
for c in countries:
    for year in years:    
        for seriescode in seriescodes_13:
            if not isinstance(dict_13[c].loc[seriescode, year], float):
                dict_13[c].loc[seriescode, year] = float(dict_13[c].loc[seriescode, year].replace(',', '').replace('<', '').replace('>', '').replace('=', '').replace('N', '0').replace(' -   ', '0').replace('0V', '0').replace('. . .', '0'))

In [41]:
# double-check: are all series codes as rows?
len(list(dict_13['Nicaragua'].index))

14

Save information file.

In [42]:
info.to_csv(r'/Users/saravallejomengod/MathsYear4/M4R/utils/info.csv')

## All SDGs

We now go back to considering all SDGs. 

Let's load the wb_info data. This contains meta-information about the series codes, mapping them to the indicators, taargets and goals.

In [43]:
wb_info_new = pd.read_csv('/Users/saravallejomengod/MathsYear4/M4R/utils/wb_info.csv', header=None, dtype=object)
print(len(wb_info_new))

401


In [44]:
wb_info_new.head()

,0,1,2,3,4,5
0,per_lm_alllm.cov_pop_tot,NaN,Coverage of unemployment benefits and ALMP (% ...,1,1.3,1
1,per_lm_alllm.cov_q1_tot,NaN,Coverage of unemployment benefits and ALMP in ...,1,1.3,1
2,per_lm_alllm.cov_q2_tot,NaN,Coverage of unemployment benefits and ALMP in ...,1,1.3,1
3,per_lm_alllm.cov_q3_tot,NaN,Coverage of unemployment benefits and ALMP in ...,1,1.3,1
4,per_lm_alllm.cov_q4_tot,NaN,Coverage of unemployment benefits and ALMP in ...,1,1.3,1


In [45]:
# the percentage of targets we have data for
print(round(len(wb_info_new[4].unique())/169, 4)*100, '%')

45.56 %


In [46]:
# are any indicators double?
wb_info_new[wb_info_new.duplicated(subset=[2])==True]

,0,1,2,3,4,5
340,DT.ODA.ODAT.CD,Economic Policy & Debt: Official development a...,Net official development assistance received (...,17,17.2,1


In [47]:
# drop this indicator
wb_info_new.drop_duplicates(subset=[2], inplace=True)
print(len(wb_info_new))

400


In [48]:
wb_info_new[wb_info_new[3]=='13']

,0,1,2,3,4,5
310,EN.CLC.DRSK.XQ,Environment: Land use,Disaster risk reduction progress score (1-5 sc...,13,13.2,-1
311,EN.CLC.MDAT.ZS,Environment: Land use,"Droughts, floods, extreme temperatures (% of p...",13,13.1,-1
388,SG_DSR_SILN,NaN,Number of local governments that adopt and imp...,13,13.1,1
389,SG_DSR_SILS,NaN,Proportion of local governments that adopt and...,13,13.1,1
390,SG_GOV_LOGV,NaN,Number of local governments (number),13,13.1,1
391,VC_DSR_AFFCT,NaN,Number of people affected by disaster (number),13,13.1,-1
392,VC_DSR_DAFF,NaN,Number of directly affected persons attributed...,13,13.1,-1
393,VC_DSR_IJILN,NaN,Number of injured or ill people attributed to ...,13,13.1,-1
394,VC_DSR_MISS,NaN,Number of missing persons due to disaster (num...,13,13.1,-1
395,VC_DSR_MORT,NaN,Number of deaths due to disaster (number),13,13.1,-1


We convert the data set into multiple small data sets by creating a dictionary that contains the Country names as keys. We add to this the data for SDG 13 from UN.

Each column will be the year and the rows will correspond to the Series Codes.

In [49]:
dict_all_wb = {country: pd.DataFrame() for country in countries}
for country in countries:
    print(country)
    dict_all_wb[country] = wb_data[wb_data['Country Name'].isin(['{}'.format(country)])]
    dict_all_wb[country] = dict_all_wb[country].drop(columns=['Country Name', 'Country Code', 'Series Name'])
    dict_all_wb[country].set_index('Series Code', inplace=True) #set series code as index
    dict_all_wb[country] = pd.concat([dict_all_wb[country], dict_13[country]])    # adding series codes for SDG 13
    dict_all_wb[country] = dict_all_wb[country].replace('..', np.nan).astype(float)
    dict_all_wb[country] = dict_all_wb[country].drop(index='DT.ODA.ODAT.CD')

Afghanistan
Albania
Algeria
Angola
Antigua and Barbuda
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas, The
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Comoros
Congo, Dem. Rep.
Congo, Rep.
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Cyprus
Czech Republic
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt, Arab Rep.
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Fiji
Finland
France
Gabon
Gambia, The
Georgia
Germany
Ghana
Greece
Grenada
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran, Islamic Rep.
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
Korea, Rep.
Kuwait
Kyrgyz Republic
Lao PDR
Latvia
Lebanon
Lesotho
Liberia
Libya
Lithuania
Luxembourg
Madagascar
Malawi
Malaysia
Maldives
Mali
Malta
Mauritania
Mauritius
Mexico


In [50]:
dict_all_wb['Korea, Rep.']

TimePeriod,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96.800000,96.680000,96.790000,96.800000,96.770000,96.68000,96.68000,NaN,NaN,NaN
EG.ELC.ACCS.ZS,99.979927,99.989868,99.996658,99.999481,100.0,100.0,100.0,100.0,100.0,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.00000,100.000000,100.00000,NaN
EG.ELC.ACCS.RU.ZS,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.00000,100.000000,100.00000,NaN
EG.ELC.ACCS.UR.ZS,99.935440,99.971169,99.992416,99.999176,100.0,100.0,100.0,100.0,100.0,99.996605,...,99.939232,99.974922,99.993843,99.999374,100.000000,100.00000,100.00000,100.000000,100.00000,NaN
FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,93.046867,NaN,NaN,94.360817,NaN,NaN,94.851013,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VC_DSR_MTMN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.000000,81.000000,21.000000,16.000000,413.000000,67.00000,26.00000,72.000000,146.00000,NaN
VC_DSR_MTMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.203790,0.166730,0.043230,0.032940,0.850140,0.13119,0.05071,0.140020,0.28278,NaN
VC_DSR_PDAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96612.000000,86994.000000,26419.000000,5458.000000,10948.000000,404.00000,9035.00000,20181.000000,13399.00000,NaN
VC_DSR_PDYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,138.000000,475.000000,720.000000,32.000000,52.000000,5.00000,67.00000,1800.000000,29.00000,NaN


In [51]:
seriescodes_wb = set(list(dict_all_wb['Germany'].index))

In [52]:
# saving data
for country in countries: 
    dict_all_wb[country].to_csv(r'/Users/saravallejomengod/MathsYear4/M4R/utils/Data_countries/{}_wb.csv'.format(country))
    
# as one pickle file
dictall = open('/Users/saravallejomengod/MathsYear4/M4R/utils/Data/dict_all_wb.pkl', 'wb')
pickle.dump(dict_all_wb, dictall)
dictall.close()

### Data Standarisation

In [53]:
dict_all_wb_std = copy.deepcopy(dict_all_wb)

for country in countries:
    for seriescode in seriescodes_wb:
        # adding noise as representative for measurement errors
        #noise = np.random.normal(scale=0.1, size=len(dict_all_wb[country].loc[seriescode]))
        
        #dict_all_wb[country].loc[seriescode] = dict_all_wb[country].loc[seriescode] + noise
        
        dict_all_wb_std[country].loc[seriescode] = scale(dict_all_wb[country].loc[seriescode])

In [54]:
#check
print('Original value', dict_all_wb['Belgium'].loc['ER.H2O.FWTL.ZS'])
print('-------')
print('Standardised value', dict_all_wb_std['Belgium'].loc['ER.H2O.FWTL.ZS'])

Original value 1990          NaN
1991          NaN
1992          NaN
1993          NaN
1994          NaN
1995          NaN
1996          NaN
1997    64.083333
1998          NaN
1999          NaN
2000          NaN
2001          NaN
2002    56.125000
2003          NaN
2004          NaN
2005          NaN
2006          NaN
2007    51.783333
2008          NaN
2009          NaN
2010          NaN
2011          NaN
2012    50.016667
2013          NaN
2014          NaN
2015          NaN
2016          NaN
2017          NaN
2018          NaN
2019          NaN
Name: ER.H2O.FWTL.ZS, dtype: float64
-------
Standardised value 1990         NaN
1991         NaN
1992         NaN
1993         NaN
1994         NaN
1995         NaN
1996         NaN
1997    1.580306
1998         NaN
1999         NaN
2000         NaN
2001         NaN
2002    0.114715
2003         NaN
2004         NaN
2005         NaN
2006         NaN
2007   -0.684838
2008         NaN
2009         NaN
2010         NaN
2011         NaN
2012   

In [55]:
for country in countries:
    dict_all_wb_std[country].to_csv(r'/Users/saravallejomengod/MathsYear4/M4R/utils/Data_countries_std/{}_wb.csv'.format(country))

# as one pickle file
stand = open('/Users/saravallejomengod/MathsYear4/M4R/utils/Data/dict_all_wb_std.pkl', 'wb')
pickle.dump(dict_all_wb_std, stand)
stand.close()